In [3]:
import pandas as pd

In [7]:
cols = ['Start_Time', 'End_Time', 'Distance(mi)', 'County', 'State', 'Weather_Condition', 'Sunrise_Sunset']
df = pd.read_csv('data/raw_data.csv', usecols=cols)

In [25]:
weather_df = pd.read_csv('data/weather_conditions.csv')

weather_cats = list(weather_df.columns)[1:]
weather_dict = {}

for i in range(len(weather_df)):
    
    row = weather_df.iloc[i,:]
    
    weather_dict[row['Original']] = [wc for wc in weather_cats if row[wc] == 'X']
    
weather_dict

{'Fair': [],
 'Mostly Cloudy': ['Cloudy'],
 'Cloudy': ['Cloudy'],
 'Partly Cloudy': ['Cloudy'],
 'Clear': [],
 'Light Rain': ['Rain'],
 'Overcast': ['Cloudy'],
 nan: [],
 'Scattered Clouds': ['Cloudy'],
 'Light Snow': ['Snow'],
 'Fog': ['Visibility'],
 'Haze': ['Precipitation'],
 'Rain': ['Rain'],
 'Fair / Windy': ['Windy'],
 'Heavy Rain': ['Rain'],
 'Smoke': ['Visibility'],
 'Light Drizzle': ['Rain'],
 'Thunder in the Vicinity': ['Thunder'],
 'Cloudy / Windy': ['Cloudy', 'Windy'],
 'T-Storm': ['Rain', 'Thunder', 'Windy'],
 'Mostly Cloudy / Windy': ['Cloudy', 'Windy'],
 'Thunder': ['Thunder'],
 'Snow': ['Snow'],
 'Light Rain with Thunder': ['Rain', 'Thunder'],
 'Partly Cloudy / Windy': ['Cloudy', 'Windy'],
 'Wintry Mix': ['Rain', 'Snow'],
 'Heavy T-Storm': ['Rain', 'Thunder', 'Windy'],
 'Light Rain / Windy': ['Rain', 'Visibility'],
 'Light Snow / Windy': ['Snow', 'Windy'],
 'Drizzle': ['Rain'],
 'Heavy Snow': ['Snow'],
 'Rain / Windy': ['Rain', 'Windy'],
 'Light Thunderstorms and Rain'

In [48]:
states = {
    'AK': 'Alaska',
    'AL': 'Alabama',
    'AR': 'Arkansas',
    'AZ': 'Arizona',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DC': 'District of Columbia',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'HI': 'Hawaii',
    'IA': 'Iowa',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'MA': 'Massachusetts',
    'MD': 'Maryland',
    'ME': 'Maine',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MO': 'Missouri',
    'MS': 'Mississippi',
    'MT': 'Montana',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'NE': 'Nebraska',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NV': 'Nevada',
    'NY': 'New York',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VA': 'Virginia',
    'VT': 'Vermont',
    'WA': 'Washington',
    'WI': 'Wisconsin',
    'WV': 'West Virginia',
    'WY': 'Wyoming'
}

In [26]:
df.head()

,Start_Time,End_Time,Distance(mi),County,State,Weather_Condition,Sunrise_Sunset
0,2016-02-08 00:37:08,2016-02-08 06:37:08,3.230,Franklin,OH,Light Rain,Night
1,2016-02-08 05:56:20,2016-02-08 11:56:20,0.747,Montgomery,OH,Light Rain,Night
2,2016-02-08 06:15:39,2016-02-08 12:15:39,0.055,Hamilton,OH,Overcast,Night
3,2016-02-08 06:51:45,2016-02-08 12:51:45,0.123,Summit,OH,Overcast,Night
4,2016-02-08 07:53:43,2016-02-08 13:53:43,0.500,Hamilton,OH,Light Rain,Day


In [52]:
# Timestamp year month day time_of_day day_of_week duration distance county state day_or_night ... weather


def time_of_day(x):
    return ('Morning' if 5 <= x.hour <= 11 
            else 'Afternoon' if 12 <= x.hour <= 12+4
            else 'Evening' if 12+5 <= x.hour <= 12+8
            else 'Night')

In [53]:
df_final = pd.DataFrame({
    'Timestamp' : df['Start_Time'],
    'Year' : pd.to_datetime(df['Start_Time']).apply(lambda x: x.year),
    'Month' : pd.to_datetime(df['Start_Time']).apply(lambda x: x.month),
    'Day' : pd.to_datetime(df['Start_Time']).apply(lambda x: x.day),
    'Time_of_Day' : pd.to_datetime(df['Start_Time']).apply(time_of_day),
    'Day_of_Week' : pd.to_datetime(df['Start_Time']).apply(lambda x: x.day_of_week),
    'Duration' : (pd.to_datetime(df['End_Time']) - pd.to_datetime(df['Start_Time'])).apply(lambda x: x.seconds // 60),
    'Distance' : df['Distance(mi)'],
    'County' : df['County'],
    'State' : df['State'].map(states),
})

for wc in weather_cats:
    df_final[wc] = df['Weather_Condition'].apply(lambda x: int(wc in weather_dict[x]))

df_final

,Timestamp,Year,Month,Day,Time_of_Day,Day_of_Week,Duration,Distance,County,State,Cloudy,Rain,Snow,Precipitation,Thunder,Visibility,Windy
0,2016-02-08 00:37:08,2016,2,8,Night,0,360,3.230,Franklin,Ohio,0,1,0,0,0,0,0
1,2016-02-08 05:56:20,2016,2,8,Morning,0,360,0.747,Montgomery,Ohio,0,1,0,0,0,0,0
2,2016-02-08 06:15:39,2016,2,8,Morning,0,360,0.055,Hamilton,Ohio,1,0,0,0,0,0,0
3,2016-02-08 06:51:45,2016,2,8,Morning,0,360,0.123,Summit,Ohio,1,0,0,0,0,0,0
4,2016-02-08 07:53:43,2016,2,8,Morning,0,360,0.500,Hamilton,Ohio,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2845337,2019-08-23 18:03:25,2019,8,23,Evening,4,28,0.543,Riverside,California,0,0,0,0,0,0,0
2845338,2019-08-23 19:11:30,2019,8,23,Evening,4,26,0.338,San Diego,California,0,0,0,0,0,0,0
2845339,2019-08-23 19:00:21,2019,8,23,Evening,4,28,0.561,Orange,California,1,0,0,0,0,0,0
2845340,2019-08-23 19:00:21,2019,8,23,Evening,4,29,0.772,Los Angeles,California,0,0,0,0,0,0,0


In [58]:
import seaborn as sns
import matplotlib.pyplot as plt

In [70]:
for wc in weather_cats:
    
    print(wc,(df_final[wc] == 1).mean())


Cloudy 0.39000338096439724
Rain 0.074649022859115
Snow 0.020327960575565258
Precipitation 0.0134082300124203
Thunder 0.011381760083673597
Visibility 0.019363577383667764
Windy 0.01840973773978664


In [56]:
df_final.to_csv('data/us_accidents.csv', index=False)

In [5]:
df_final = pd.read_csv('data/us_accidents.csv')

In [6]:
import json

In [7]:
with open('geography/counties.json', 'r') as f:
    counties_dict = json.load(f)

with open('geography/states.json', 'r') as f:
    states_dict = json.load(f)

In [8]:
state_ids = {state['properties']['NAME'] : state['properties']['STATE'] for state in states_dict['features']}
id_states = {state['properties']['STATE'] : state['properties']['NAME'] for state in states_dict['features']}

In [9]:
geo_counties = set([(county['properties']['NAME'], county['properties']['STATE']) for county in counties_dict['features']])
data_counties = set(list(zip(df_final['County'], df_final['State'].map(state_ids))))

In [10]:
list(geo_counties)[:10]

[('Leelanau', '26'),
 ('Humacao', '72'),
 ('Jackson', '13'),
 ('Stark', '38'),
 ('Somervell', '48'),
 ('Monroe', '26'),
 ('Henry', '13'),
 ('Piscataquis', '23'),
 ('Butler', '19'),
 ('Grant', '20')]

In [11]:
cs = data_counties.difference(geo_counties)

In [12]:
cs

{('Alexandria City', '51'),
 ('Baltimore City', '24'),
 ('Baltimore County', '24'),
 ('Bedford County', '51'),
 ('Buena Vista City', '51'),
 ('Charlottesville City', '51'),
 ('Chesapeake City', '51'),
 ('Covington City', '51'),
 ('Danville City', '51'),
 ('De Kalb', '01'),
 ('De Kalb', '18'),
 ('De Soto', '12'),
 ('Dekalb', '13'),
 ('Dekalb', '17'),
 ('Desoto', '28'),
 ('Dewitt', '17'),
 ('District Of Columbia', '11'),
 ('District Of Columbia', '24'),
 ('Do�a Ana', '35'),
 ('Dupage', '17'),
 ('Fairfax City', '51'),
 ('Fairfax County', '51'),
 ('Fond Du Lac', '55'),
 ('Franklin City', '51'),
 ('Franklin County', '51'),
 ('Fredericksburg City', '51'),
 ('Hampton City', '51'),
 ('Harrisonburg City', '51'),
 ('Hopewell City', '51'),
 ('Isle Of Wight', '51'),
 ('Kenosha', '17'),
 ('King And Queen', '51'),
 ('La Porte', '18'),
 ('La Salle', '17'),
 ('Lagrange', '18'),
 ('Lewis And Clark', '30'),
 ('Lynchburg City', '51'),
 ('Macon-Bibb County', '13'),
 ('Manassas City', '51'),
 ('Martinsvill

In [13]:
def dist(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]

In [14]:
c_map = {}

for c, s in cs:
    c_map[(c, s)] = min([((county, state), dist(c, county)) for county, state  in list(geo_counties) if state == s], key=lambda x: x[1])[0]

c_map[('Hampton City', '51')] = ('Hampton', '51')
c_map[('Macon-Bibb County', '13')] = ('Macon', '13')
c_map[('District Of Columbia', '24')] = ('District of Columbia', '11')
c_map[('Walworth', '17')] = ('Walworth', '55')
c_map[('Kenosha', '17')] = ('Kenosha', '55')

removes = [('Oglala Lakota', '46')]

for r in removes:
    del c_map[r]
    
c_map

{('Mcleod', '27'): ('McLeod', '27'),
 ('Roanoke City', '51'): ('Roanoke', '51'),
 ('La Salle', '17'): ('LaSalle', '17'),
 ('Fairfax County', '51'): ('Fairfax', '51'),
 ('De Kalb', '01'): ('DeKalb', '01'),
 ('Saint Tammany', '22'): ('St. Tammany', '22'),
 ('Prince Georges', '24'): ("Prince George's", '24'),
 ('Mccracken', '21'): ('McCracken', '21'),
 ('Norfolk City', '51'): ('Norfolk', '51'),
 ('Fairfax City', '51'): ('Fairfax', '51'),
 ('St. Louis City', '29'): ('St. Louis', '29'),
 ('Saint Bernard', '22'): ('St. Bernard', '22'),
 ('Walworth', '17'): ('Walworth', '55'),
 ('De Soto', '12'): ('DeSoto', '12'),
 ('Harrisonburg City', '51'): ('Harrisonburg', '51'),
 ('Saint Louis', '27'): ('St. Louis', '27'),
 ('Mchenry', '17'): ('McHenry', '17'),
 ('Mcintosh', '13'): ('McIntosh', '13'),
 ('Buena Vista City', '51'): ('Buena Vista', '51'),
 ('Mcclain', '40'): ('McClain', '40'),
 ('Isle Of Wight', '51'): ('Isle of Wight', '51'),
 ('St. Louis County', '29'): ('St. Louis', '29'),
 ('De Kalb', '

In [16]:
df_final = pd.read_csv('data/us_accidents.csv')

drop_indices = []

def update(x):
    
    c, s = x.County, x.State
    
    if (c,state_ids[s]) in c_map:
        c2, s2 = c_map[c,state_ids[s]]
        return c2, id_states[s2]
    
    return c, s

new_entries = df_final.apply(update, axis=1)
df_final['County'] = new_entries.apply(lambda x: x[0])
df_final['State'] = new_entries.apply(lambda x: x[1])

In [19]:
df_final = df_final.drop(index=[2509171])

In [20]:
set(list(zip(df_final['County'], df_final['State'].map(state_ids)))).difference(geo_counties)

set()

In [94]:
df_final

,Timestamp,Year,Month,Day,Time_of_Day,Day_of_Week,Duration,Distance,County,State,Cloudy,Rain,Snow,Precipitation,Thunder,Visibility,Windy
0,2016-02-08 00:37:08,2016,2,8,Night,0,360,3.230,Franklin,Ohio,0,1,0,0,0,0,0
1,2016-02-08 05:56:20,2016,2,8,Morning,0,360,0.747,Montgomery,Ohio,0,1,0,0,0,0,0
2,2016-02-08 06:15:39,2016,2,8,Morning,0,360,0.055,Hamilton,Ohio,1,0,0,0,0,0,0
3,2016-02-08 06:51:45,2016,2,8,Morning,0,360,0.123,Summit,Ohio,1,0,0,0,0,0,0
4,2016-02-08 07:53:43,2016,2,8,Morning,0,360,0.500,Hamilton,Ohio,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2845337,2019-08-23 18:03:25,2019,8,23,Evening,4,28,0.543,Riverside,California,0,0,0,0,0,0,0
2845338,2019-08-23 19:11:30,2019,8,23,Evening,4,26,0.338,San Diego,California,0,0,0,0,0,0,0
2845339,2019-08-23 19:00:21,2019,8,23,Evening,4,28,0.561,Orange,California,1,0,0,0,0,0,0
2845340,2019-08-23 19:00:21,2019,8,23,Evening,4,29,0.772,Los Angeles,California,0,0,0,0,0,0,0


In [21]:
states = {state : i for i, state in enumerate(sorted(list(set(df_final['State']))))}
counties = {county : i for i, county in enumerate(sorted(list(set(df_final['County']))))}
tods = {'Morning' : 0, 'Afternoon' : 1, 'Evening' : 2, 'Night' : 3}

In [25]:
regions_dict = {'states' : list(states.keys()), 'counties' : list(counties.keys())}
with open('data/regions.js', 'w') as f:
    json.dump(regions_dict, f)

In [100]:
df_comp = pd.DataFrame(df_final)

df_comp['Time_of_Day'] = df_comp['Time_of_Day'].map(tods)
df_comp['County'] = df_comp['County'].map(counties)
df_comp['State'] = df_comp['State'].map(states)

In [101]:
df_comp

,Timestamp,Year,Month,Day,Time_of_Day,Day_of_Week,Duration,Distance,County,State,Cloudy,Rain,Snow,Precipitation,Thunder,Visibility,Windy
0,2016-02-08 00:37:08,2016,2,8,3,0,360,3.230,534,33,0,1,0,0,0,0,0
1,2016-02-08 05:56:20,2016,2,8,0,0,360,0.747,996,33,0,1,0,0,0,0,0
2,2016-02-08 06:15:39,2016,2,8,0,0,360,0.055,636,33,1,0,0,0,0,0,0
3,2016-02-08 06:51:45,2016,2,8,0,0,360,0.123,1414,33,1,0,0,0,0,0,0
4,2016-02-08 07:53:43,2016,2,8,0,0,360,0.500,636,33,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2845337,2019-08-23 18:03:25,2019,8,23,2,4,28,0.543,1246,3,0,0,0,0,0,0,0
2845338,2019-08-23 19:11:30,2019,8,23,2,4,26,0.338,1286,3,0,0,0,0,0,0,0
2845339,2019-08-23 19:00:21,2019,8,23,2,4,28,0.561,1089,3,1,0,0,0,0,0,0
2845340,2019-08-23 19:00:21,2019,8,23,2,4,29,0.772,872,3,0,0,0,0,0,0,0


In [102]:
df_comp.to_csv('data/compressed.csv', index=False)

In [1]:
counties

NameError: name 'counties' is not defined

In [26]:
states

{'Alabama': 0,
 'Arizona': 1,
 'Arkansas': 2,
 'California': 3,
 'Colorado': 4,
 'Connecticut': 5,
 'Delaware': 6,
 'District of Columbia': 7,
 'Florida': 8,
 'Georgia': 9,
 'Idaho': 10,
 'Illinois': 11,
 'Indiana': 12,
 'Iowa': 13,
 'Kansas': 14,
 'Kentucky': 15,
 'Louisiana': 16,
 'Maine': 17,
 'Maryland': 18,
 'Massachusetts': 19,
 'Michigan': 20,
 'Minnesota': 21,
 'Mississippi': 22,
 'Missouri': 23,
 'Montana': 24,
 'Nebraska': 25,
 'Nevada': 26,
 'New Hampshire': 27,
 'New Jersey': 28,
 'New Mexico': 29,
 'New York': 30,
 'North Carolina': 31,
 'North Dakota': 32,
 'Ohio': 33,
 'Oklahoma': 34,
 'Oregon': 35,
 'Pennsylvania': 36,
 'Rhode Island': 37,
 'South Carolina': 38,
 'South Dakota': 39,
 'Tennessee': 40,
 'Texas': 41,
 'Utah': 42,
 'Vermont': 43,
 'Virginia': 44,
 'Washington': 45,
 'West Virginia': 46,
 'Wisconsin': 47,
 'Wyoming': 48}